<a href="https://colab.research.google.com/github/Ananya-AJ/CMPE255-Data-Mining/blob/main/ANN/ANN_LSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d benhamner/nips-papers

 86% 121M/141M [00:01<00:00, 102MB/s]
100% 141M/141M [00:01<00:00, 109MB/s]


In [ ]:
!unzip nips-papers.zip

Archive:  nips-papers.zip
  inflating: authors.csv             
  inflating: database.sqlite         
  inflating: paper_authors.csv       
  inflating: papers.csv              


In [ ]:
!pip install datasketch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 3.6 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

##split text into tokens/shingles based on whitespace

In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [ ]:
#Number of Permutations
permutations = 128

#Number of Recommendations to return
num_recommendations = 1

Pass in a dataframe with every string you want to query.
Preprocess a string of text using our preprocessing step above.
Set the number of permutations in your MinHash.
MinHash the string on all of your shingles in the string.
Store the MinHash of the string.
Repeat 2-5 for all strings in your dataframe.
Build a forest of all the MinHashed strings.
Index your forest to make it searchable.

In [ ]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

Preprocess your text into shingles.
Set the same number of permutations for your MinHash as was used to build the forest.
Create your MinHash on the text using all your shingles.
Query the forest with your MinHash and return the number of requested recommendations.
Provide the titles of each conference paper recommended.


In [ ]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

Testing on NIPS

In [ ]:
db = pd.read_csv('papers.csv')
db['text'] = db['title'] + ' ' + db['abstract']
forest = get_forest(db, permutations)

It took 16.60693335533142 seconds to build forest.


In [ ]:
num_recommendations = 5
title = 'Using LSH to build recommendation system'
result = predict(title,db, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.013130664825439453 seconds to query forest.

 Top Recommendation(s) is(are) 
 2592     Random Sampling of States in Dynamic Programming
3242    A rational decision making framework for inhib...
2415             Random Projections for Manifold Learning
342     Statistically Efficient Estimations Using Cort...
4764    The Bayesian Case Model: A Generative Approach...
Name: title, dtype: object
